In [1]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import unicodedata
import re

In [2]:
data=pd.read_excel('赛题1数据材料/赛题1数据集/专利.xlsx',index_col=0)

In [3]:
data

,专利类型,授权公告日,申请日
企业编号,,,
1001,发明公布,2006-09-08­,2008-03-12
1001,发明公布,2006-06-23­,2007-12-26
1001,发明公布,2006-06-23,2007-12-26
1001,发明公布,2006-09-08,2008-03-12
1002,实用新型,2010-09-26,2011-10-26
1002,发明公布,2010-04-16,2010-09-22
1003,实用新型,2014-11-28,2015-04-22
1003,发明公布,2012-09-18,2012-12-19
1003,实用新型,2009-06-09,2010-04-14


In [4]:
data.drop_duplicates(keep='first',inplace=True)

In [5]:
data=data.replace(['发明公布更正','实用新型更正','外观设计更正','发明授权更正'],np.nan)
data=data.dropna()
count=pd.DataFrame(data.index.value_counts()).T.reindex(columns=list(range(1001,4001))).fillna(0)
count.index=pd.Index(['专利_count'])
data_1=pd.get_dummies(data['专利类型'],prefix='专利类型')
data_1=data_1.groupby('企业编号').sum().T

In [6]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d.+\d', cell)
    if not _:
        return None
    cell = _[0]
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [7]:
data_2=data[['授权公告日']].applymap(to_day).groupby('企业编号').mean().T
data_3=data[['申请日']].applymap(to_day).groupby('企业编号').mean().T

In [9]:
data=pd.concat([data_1,data_2,data_3,count])
data=data.reindex(columns=list(range(1001,4001))).fillna(0)
data.to_pickle('专利.pickle')